Gasto de las ligas a lo largo de los años:

https://www.statista.com/statistics/1022140/european-soccer-wage-costs-by-league/

Tasa de inflación pasada:

https://fxtop.com/es/convertidor-divisas-pasado.php?A=100&C1=EUR&C2=EUR&DD=16&MM=11&YYYY=2015&B=1&P=&I=1&btnOK=Ir 

In [1]:
years = list(range(2017, 2023))

In [2]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd

Scrapeamos estadísticas de jugadores.

In [62]:
url_start = ['https://fbref.com/es/comps/Big5/{}-{}/stats/jugadores/Estadisticas-{}-{}-Las-5-grandes-ligas-europeas',
             'https://fbref.com/es/comps/Big5/{}-{}/keepers/jugadores/Estadisticas-{}-{}-Las-5-grandes-ligas-europeas',
             'https://fbref.com/es/comps/Big5/{}-{}/misc/jugadores/Estadisticas-{}-{}-Las-5-grandes-ligas-europeas',
             'https://fbref.com/es/comps/Big5/{}-{}/passing/jugadores/Estadisticas-{}-{}-Las-5-grandes-ligas-europeas',
             'https://fbref.com/es/comps/Big5/{}-{}/possession/jugadores/Estadisticas-{}-{}-Las-5-grandes-ligas-europeas',
             'https://fbref.com/es/comps/Big5/{}-{}/shooting/jugadores/Estadisticas-{}-{}-Las-5-grandes-ligas-europeas']
dir_names = ['stats', 'porteros', 'masStats', 'pases', 'posesion', 'tiros']

driver = webdriver.Chrome()

for i in range(len(dir_names)):
    for year in years:
        url = url_start[i].format(year, year+1, year, year+1)

        driver.get(url)
        driver.execute_script("window.scrollTo(1, 10000)")
        time.sleep(2)

        html_code = driver.page_source

        with open(("html/"+dir_names[i]+"/{}-{}.html").format(year, year+1), "w+", encoding='utf-8') as f:
            f.write(html_code)



In [64]:
id = ['stats_standard', 'stats_keeper', 'stats_misc', 'stats_passing', 'stats_possession', 'stats_shooting']

for i in range(len(id)):
    dfs = []
    
    for year in years:
        with open(("html/"+dir_names[i]+"/{}-{}.html").format(year, year+1), encoding='utf-8') as f:
            page = f.read()

        soup = BeautifulSoup(page, "html.parser")
        
        # Encuentra y elimina todas las filas de cabecera superior en la tabla
        for header_row in soup.find_all('tr', class_="over_header"):
            header_row.decompose()

        # Encuentra y elimina todas las filas de cabecera repetidas en la tabla
        for header_row in soup.find_all('tr', class_="thead"):
            header_row.decompose()

        # Convierte el HTML a un Dataframe
        stats_table = soup.find(id=id[i])
        stats_table = pd.read_html(str(stats_table))[0]
        stats_table["Temporada"] = year # Añade el año en que empieza la temporada de dichas estadísticas

        dfs.append(stats_table)
    
    stats = pd.concat(dfs) # Concatenamos todos los frames de x estadística
    stats.to_csv("csv/"+id[i]+".csv", index=False) # Guardamos en formato .csv


C:\Users\34667\AppData\Local\Temp\ipykernel_12304\2853051243.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_table = pd.read_html(str(stats_table))[0]
C:\Users\34667\AppData\Local\Temp\ipykernel_12304\2853051243.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_table = pd.read_html(str(stats_table))[0]
C:\Users\34667\AppData\Local\Temp\ipykernel_12304\2853051243.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_table = pd.read_html(str(stats_table))[0]
C:\Users\34667\AppData\Local\Temp\ipykernel_12304\2853051243.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and wi

Scrapeamos salarios de jugadores.

In [5]:
url_start = 'https://fbref.com/es/comps/{}/{}-{}/wages/{}-{}-{}-Salarios'
comp = ['9', '12', '20', '13', '11']
liga = ['Premier-League', 'La-Liga', 'Bundesliga', 'Ligue-1', 'Serie-A']

driver = webdriver.Chrome()

for lig in range(len(liga)):
    for year in years:
        url = url_start.format(comp[lig], year+1, year+2, liga[lig], year+1, year+2)

        driver.get(url)
        driver.execute_script("window.scrollTo(1, 10000)")
        time.sleep(2)

        html_code = driver.page_source

        with open(("html/salary/{}-{}-{}.html").format(liga[lig], year+1, year+2), "w+", encoding='utf-8') as f:
            f.write(html_code)

In [10]:
dfs = []

for lig in range(len(liga)):
    for year in years:
        with open(("html/salary/{}-{}-{}.html").format(liga[lig], year+1, year+2), encoding='utf-8') as f:
            page = f.read()

        soup = BeautifulSoup(page, "html.parser")
        
        # Encuentra y elimina todas las filas de cabecera repetidas en la tabla
        for header_row in soup.find_all('tr', class_="thead"):
            header_row.decompose()

        # Convierte el HTML a un Dataframe
        stats_table = soup.find(id='player_wages')
        stats_table = pd.read_html(str(stats_table))[0]
        stats_table["TempAnterior"] = year # Añade el año equivalente al que pertenecen las etiquetas (características del año anterior)
        stats_table["Comp"] = liga[lig] # Añade la competición en la que juega el jugador

        dfs.append(stats_table)
    
stats = pd.concat(dfs) # Concatenamos todos los dataframes
stats.to_csv("csv/salarios.csv", index=False) # Guardamos en formato .csv


C:\Users\34667\AppData\Local\Temp\ipykernel_11356\2610223657.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_table = pd.read_html(str(stats_table))[0]
C:\Users\34667\AppData\Local\Temp\ipykernel_11356\2610223657.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_table = pd.read_html(str(stats_table))[0]
C:\Users\34667\AppData\Local\Temp\ipykernel_11356\2610223657.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_table = pd.read_html(str(stats_table))[0]
C:\Users\34667\AppData\Local\Temp\ipykernel_11356\2610223657.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and wi

Scrapeamos clasificación de los equipos.

In [7]:
url_start = 'https://fbref.com/es/comps/Big5/{}-{}/Estadisticas-{}-{}-Las-5-grandes-ligas-europeas'

driver = webdriver.Chrome()

for year in years:
    url = url_start.format(year, year+1, year, year+1)

    driver.get(url)
    driver.execute_script("window.scrollTo(1, 10000)")
    time.sleep(2)

    html_code = driver.page_source

    with open(("html/clasificacion/{}-{}.html").format(year, year+1), "w+", encoding='utf-8') as f:
        f.write(html_code)

In [9]:
dfs = []

for year in years:
    with open(("html/clasificacion/{}-{}.html").format(year, year+1), encoding='utf-8') as f:
        page = f.read()

    soup = BeautifulSoup(page, "html.parser")
    
    # Encuentra y elimina todas las filas de cabecera repetidas en la tabla
    for header_row in soup.find_all('tr', class_="thead"):
        header_row.decompose()

    # Convierte el HTML a un Dataframe
    stats_table = soup.find(id='big5_table')
    stats_table = pd.read_html(str(stats_table))[0]
    stats_table["Temporada"] = year # Añade el año en que empieza la temporada de dichas clasificaciones

    dfs.append(stats_table)
    
stats = pd.concat(dfs) # Concatenamos todos los dataframes
stats.to_csv("csv/clasificacion.csv", index=False) # Guardamos en formato .csv




C:\Users\34667\AppData\Local\Temp\ipykernel_11356\902674581.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_table = pd.read_html(str(stats_table))[0]
C:\Users\34667\AppData\Local\Temp\ipykernel_11356\902674581.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_table = pd.read_html(str(stats_table))[0]
C:\Users\34667\AppData\Local\Temp\ipykernel_11356\902674581.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_table = pd.read_html(str(stats_table))[0]
C:\Users\34667\AppData\Local\Temp\ipykernel_11356\902674581.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will b